# Clustering Crypto

In [26]:
# Initial imports
import requests
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [27]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"


In [28]:
# Alternatively, use the provided csv file:

# Create a DataFrame


In [29]:
# Create a funtion to read in DataFrame 
def read_json(url):
    url = "https://min-api.cryptocompare.com/data/all/coinlist"
    request = Request(url)
    response = urlopen(request)
    data = response.read()
    url_data = json.loads(data)
    return(url_data)
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_df = pd.DataFrame(read_json(url)["Data"]).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,SortOrder,Sponsored,Taxonomy,Rating,IsTrading,TotalCoinsMined,BlockNumber,NetHashesPerSecond,BlockReward,BlockTime
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,34,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,42,204348,0,0,0
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2212,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,300,0,0,0,0
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,916,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,602,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,1.9404e+06,64888,0,34.8398,60
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,3505,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",False,1.12518e+08,10821438,231060807658772,2,0


### Data Preprocessing

In [30]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName',
           'Algorithm',
           'IsTrading',
           'ProofType',
           'TotalCoinsMined',
           'TotalCoinSupply']]

In [31]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [32]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df["Algorithm"] != "N/A"]

In [33]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns = "IsTrading")

In [34]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [35]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df["TotalCoinsMined"] != 0]

In [36]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.loc[crypto_df["TotalCoinSupply"] != 'N/A']
crypto_df = crypto_df.loc[crypto_df["ProofType"] != 'N/A']


In [37]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df["CoinName"])
coins_name

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
ETH,Ethereum
NXT,Nxt
...,...
DCR,Decred
STEEM,Steem
BTC,Bitcoin
BUX,Buxcoin


In [38]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns = "CoinName")
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.9404e+06,532000000
1337,X13,PoW/PoS,2.95049e+10,314159265359
ETH,Ethash,PoW,1.12518e+08,0
NXT,PoS,PoS/LPoS,1000000000,1000000000
...,...,...,...,...
DCR,BLAKE256,PoW/PoS,1.2182e+07,21000000
STEEM,PoS,PoW,99692126,99692126
BTC,SHA-256,PoW,18528943,21000000
BUX,Scrypt,PoW/PoS,4.50065e+08,500000000


In [ ]:
# Create dummy variables for text features


In [ ]:
# Standardize data


### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components


In [ ]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [84]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(new_crypto_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

ValueError: could not convert string to float: 'PoS'

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
